"""
    
    '{06-2022-3-2}'
    
    This will convert IAM data into toy training data
    
    
    The requirment in experiment '{05-2022-28-1}' were changed so this is new 
    
    experimtn continuation of it
    
    
    It has following words coordinates
    
    
    {'the': 0,
     'of': 1,
     'to': 2,
     'and': 3,
     'in': 4,
     'was': 5,
     'that': 6,
     'is': 7,
     'he': 8,
     'for': 9,
     'with': 10,
     'his': 11,
     'it': 12,
     'which': 13,
     'they': 14,
     'from': 15,
     'are': 16,
     'been': 17,
     'you': 18,
     'this': 19}

"""


	 unique: 1539


"""
    
    
    #filter out only images which are into train split of IAM handwritten.

"""

In [2]:


import os
splitFilePath="./data/largeWriterIndependentTextLineRecognitionTask"

#os.listdir(splitFilePath)

trainFile=os.path.join(splitFilePath, 'trainset.txt')
testFile=os.path.join(splitFilePath,'testset.txt')
validFile1=os.path.join(splitFilePath,'validationset1.txt')
validFile2=os.path.join(splitFilePath,'validationset2.txt')

"""
    below function takes the line number and finds to which original image it 
    belongs, this image split will help to identify yolo train,test,valid data
    
"""
def gatherFormInOfficialSplitForYolo(temp):

    formsPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//"

    gatherList=[]
    allLines=[]
    
    for indx,lineDetail in enumerate(temp):

        count=0
        imgNameIndx=0
        lineNoIndx=0
        #print(" lineDetail:",lineDetail)
        for charIndx,ele in enumerate(lineDetail):

            if ele=="-":
                count+=1
                
            if count==2:
                imgNameIndx=charIndx
                break
            if count==3:
                lineNoIndx=charIndx
                break

        pngName=lineDetail[:imgNameIndx]+".png"
        pngPath=os.path.join(formsPath,pngName)
        
        lineName=lineDetail#[:(imgNameIndx+2)]
        allLines.append(lineName)
        
        if os.path.isfile(pngPath):
            gatherList.append(pngName)

        #print(" check File:",os.path.isfile(pngPath))
    
    gatherList=list(set(gatherList))
    allLines=list(set(allLines))
    print(" total png gathered:",len(gatherList)," original lines:",len(temp),"unique lines:",len(allLines))    

    return gatherList,allLines    


with open(trainFile) as f1:
    trainSet=f1.readlines()

trainList,trainLines=gatherFormInOfficialSplitForYolo(trainSet)

print(" train len:",len(trainList))
print(" trainLines len:",len(trainLines))


with open(testFile) as f1:
    testSet=f1.readlines()

testList,testLines=gatherFormInOfficialSplitForYolo(testSet)
    
with open(validFile1) as f1:
    validSet1=f1.readlines()

validList1,valLines1=gatherFormInOfficialSplitForYolo(validSet1)

with open(validFile2) as f1:
    validSet2=f1.readlines()

validList2,valLines2=gatherFormInOfficialSplitForYolo(validSet2)
    
validList=validList1+validList2
valLines=valLines1+valLines2


print(" test len:",len(testList))
print(" testLines len:",len(testLines))

print(" valid len:",len(validList))
print(" valLines len:",len(valLines))



 total png gathered: 747  original lines: 6161 unique lines: 6161
 train len: 747
 trainLines len: 6161
 total png gathered: 232  original lines: 1861 unique lines: 1861
 total png gathered: 105  original lines: 900 unique lines: 900
 total png gathered: 115  original lines: 940 unique lines: 940
 test len: 232
 testLines len: 1861
 valid len: 220
 valLines len: 1840


In [4]:
import pandas as pd

df=pd.read_csv("./data/data_14_april.csv")
unqImages=set(df.image_name)
print("\n\t unique:",len(unqImages))


	 unique: 1539


In [18]:

print(df.columns)
freqDist=df['text'].value_counts()
freqDict=freqDist.to_dict()

Index(['image_name', 'class', 'width', 'height', 'org_x1', 'org_y1', 'org_x2',
       'org_y2', 'text', 'cropName', 'x', 'y', 'w', 'h'],
      dtype='object')


In [23]:
len(list(freqDict.keys()))

#freqDist[:50]

13550

In [29]:
"""
    these are the filtered words for localization
"""


words=["the","of","to","and","in","was","that","is","he","for","with","his","it","which","they","from","are","been","you","this"]     

classDict=dict()


for classId,w in enumerate(words):

    classDict[w]=classId
    
#classDict


In [30]:
df.columns

Index(['image_name', 'class', 'width', 'height', 'org_x1', 'org_y1', 'org_x2',
       'org_y2', 'text', 'cropName', 'x', 'y', 'w', 'h'],
      dtype='object')

In [39]:
#for file in ['train','test']:
import numpy as np
import os

savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/toyData_#06-2022-3-2#//"
images_path = "./data/datasets/forms//"
#allImages=os.listdir(images_path)
#images_num=len(os.listdir(images_path))

#labels_txt = savePath+"/"+"mnist_train1_toyData_#06-2022-3-2#.txt"
labels_txt2 = savePath+"/"+"mnist_test1_toyData_#06-2022-3-2#.txt"

lastName=""
curLine=""
prevLine=""
unqImg=[]
exceptionCount,skipImage=0,0

for indx,info in df.iterrows():
    

    if not info['image_name'] in trainList or not info["text"] in classDict:# 
        continue
    
    """
    if not info['image_name'] in testList or not info["text"] in classDict:# 
        continue
    """
    try:
        """
        if indx%100==0:
            print("\n\t unq images:",len(unqImg))
        """
        if len(unqImg)<30000:

            with open(labels_txt, "a") as wf:
                image_path =images_path+info["image_name"]
                """
                if indx%100==0:
                    print("\n\t path:",image_path)
                """
                if info["image_name"]!=lastName and info["image_name"] not in unqImg:
                    curLine=image_path

                    if len(prevLine):
                        #print("\n\t prevLine:",prevLine)
                        #wf.write( image_path+ "\n")
                        wf.write(prevLine+"\n")

                else:
                    xmin,ymin=str(int(info["org_x1"])),str(int(info["org_y1"]))
                    xmax,ymax=str(int(info["org_x2"])),str(int(info["org_y2"]))
                    curLine += ' ' + ','.join([xmin, ymin, xmax, ymax, str(classDict[info["text"]])])
                    prevLine=curLine

                lastName=info["image_name"]

            
        if info["image_name"] not in unqImg:
            unqImg.append(info["image_name"])

    except Exception as e:
        exceptionCount+=1
        print("\n\t exception index:",indx,"\t count:",exceptionCount," e:",e)

print(" total exceptions:",exceptionCount," total unique images:",len(unqImg))

 total exceptions: 0  total unique images: 747


In [40]:
classDict

{'the': 0,
 'of': 1,
 'to': 2,
 'and': 3,
 'in': 4,
 'was': 5,
 'that': 6,
 'is': 7,
 'he': 8,
 'for': 9,
 'with': 10,
 'his': 11,
 'it': 12,
 'which': 13,
 'they': 14,
 'from': 15,
 'are': 16,
 'been': 17,
 'you': 18,
 'this': 19}